Primeiro separei os personagens com o maior número de imagens  

In [ ]:
import os
from collections import Counter
import pandas as pd
import cv2
import shutil

In [7]:
# Diretório onde estão as pastas dos personagens
diretorio_base_treino = r'C:\Users\Pichau\Desktop\Faculdade\Projects\Facul Repositorios\Repositorios_Universitarios\Redes Neurais\Projetao\simpsons_dataset'

# Obtendo todas as subpastas (personagens) e seus nomes
subpastas = [(f.path, f.name) for f in os.scandir(diretorio_base_treino) if f.is_dir()]

contagem_imagens_por_personagem = {}
for subpasta, nome_pasta in subpastas:
    arquivos = os.listdir(subpasta)
    contagem_imagens_por_personagem[nome_pasta] = len([arquivo for arquivo in arquivos if arquivo.endswith('.jpg') or arquivo.endswith('.png')])

# Ordenar as subpastas pelos números de imagens e pegar os 10 primeiros
top_10_personagens = Counter(contagem_imagens_por_personagem).most_common(10)

df = pd.DataFrame(top_10_personagens, columns=['Personagem', 'Quantidade de Imagens'])


print(df)


                 Personagem  Quantidade de Imagens
0             homer_simpson                   2246
1              ned_flanders                   1454
2               moe_szyslak                   1452
3              lisa_simpson                   1354
4              bart_simpson                   1342
5             marge_simpson                   1291
6          krusty_the_clown                   1206
7         principal_skinner                   1194
8  charles_montgomery_burns                   1193
9       milhouse_van_houten                   1079


# organisando o dataser de treino


Recortando as imagens 75x75 e rotacionando as imagens 45 e 90 graus  e selecionando apenas as imagens dos personagens que se apresentaram na lista acima 

In [ ]:
# Função para redimensionar e salvar imagens
def redimensionar_e_rotacionar_imagens(diretorio_pasta, personagem, caminho_destino):
    angulos = [45, 90]    # Ângulos de rotação desejados

    for arquivo in os.listdir(diretorio_pasta):
        if arquivo.endswith('.jpg') or arquivo.endswith('.png'):# Caminho completo da imagem original
            caminho_imagem_original = os.path.join(diretorio_pasta, arquivo)

            # Ler a imagem original
            imagem_original = cv2.imread(caminho_imagem_original)

            # Redimensionar a imagem original para 75x75
            imagem_redimensionada = cv2.resize(imagem_original, (75, 75))

            # Salvar imagem redimensionada
            caminho_destino_imagem_redimensionada = os.path.join(caminho_destino, f'{personagem}_{arquivo}')
            cv2.imwrite(caminho_destino_imagem_redimensionada, imagem_redimensionada)

            # Rotacionar e salvar a imagem em diferentes ângulos
            for angulo in angulos:
                # Rotacionar a imagem
                matriz_rotacao = cv2.getRotationMatrix2D((imagem_original.shape[1] / 2, imagem_original.shape[0] / 2), angulo, 1)
                imagem_rotacionada = cv2.warpAffine(imagem_original, matriz_rotacao, (imagem_original.shape[1], imagem_original.shape[0]))

                # Redimensionar a imagem rotacionada para 75x75
                imagem_rotacionada_redimensionada = cv2.resize(imagem_rotacionada, (75, 75))

                # Salvar a imagem rotacionada
                caminho_destino_imagem_rotacionada = os.path.join(caminho_destino, f'{personagem}_{arquivo[:-4]}_rot{angulo}.jpg')
                cv2.imwrite(caminho_destino_imagem_rotacionada, imagem_rotacionada_redimensionada)


# Diretório onde estão as pastas dos personagens
diretorio_base_treino = r'C:\Users\Pichau\Desktop\Faculdade\Projects\Facul Repositorios\Repositorios_Universitarios\Redes Neurais\Projetao\simpsons_dataset'

caminho_destino = 'dataset_treino'

# Loop sobre os personagens e suas contagens
for personagem, _ in top_10_personagens:
    # Caminho completo da pasta do personagem
    diretorio_pasta = os.path.join(diretorio_base_treino, personagem)

    # Verificar se a pasta existe
    if os.path.exists(diretorio_pasta):
        # Criar a pasta de destino se não existir
        caminho_destino_personagem = os.path.join(caminho_destino, personagem)
        os.makedirs(caminho_destino_personagem, exist_ok=True)

        redimensionar_e_rotacionar_imagens(diretorio_pasta, personagem, caminho_destino_personagem)


Corrigindo um problema do banco de testes, separando e catalogando.

In [ ]:
# Diretório onde estão as imagens misturadas
diretorio_base = r'C:\Users\Pichau\Desktop\Faculdade\Projects\Facul Repositorios\Repositorios_Universitarios\Redes Neurais\Projetao\kaggle_simpson_testset'

# Criar uma lista de todos os personagens presentes nas imagens
personagens = set()

# Listar todos os arquivos no diretório base
for arquivo in os.listdir(diretorio_base):
    # Verificar se é um arquivo de imagem
    if arquivo.endswith('.jpg') or arquivo.endswith('.png'):
        # Extrair o nome do personagem do nome do arquivo
        nome_personagem = '_'.join(arquivo.split('_')[:-1])
        # Adicionar o nome do personagem à lista de personagens
        personagens.add(nome_personagem)

for personagem in personagens: # Criar uma pasta para cada personagem
    # Caminho para a pasta do personagem
    caminho_destino = os.path.join(diretorio_base, personagem)
    # Criar a pasta se ela não existir
    os.makedirs(caminho_destino, exist_ok=True)

# Mover cada imagem para a pasta do seu respectivo personagem
for arquivo in os.listdir(diretorio_base):
    # Verificar se é um arquivo de imagem
    if arquivo.endswith('.jpg') or arquivo.endswith('.png'):
        # Extrair o nome do personagem do nome do arquivo
        nome_personagem = '_'.join(arquivo.split('_')[:-1])
        # Caminho de origem do arquivo
        caminho_origem = os.path.join(diretorio_base, arquivo)
        # Caminho de destino da pasta do personagem
        caminho_destino = os.path.join(diretorio_base, nome_personagem)
        # Mover o arquivo para a pasta do personagem
        shutil.move(caminho_origem, caminho_destino)


Montando a pasta com o conjunto de testes para os modelos

In [ ]:
# Diretório onde estão as pastas dos personagens
diretorio_base_treino = r'C:\Users\Pichau\Desktop\Faculdade\Projects\Facul Repositorios\Repositorios_Universitarios\Redes Neurais\Projetao\simpson_testset'

# Diretório de destino para as imagens redimensionadas e rotacionadas
caminho_destino = 'testset_simpsons'

# Loop sobre os personagens e suas contagens
for personagem, _ in top_10_personagens:
    # Caminho completo da pasta do personagem
    diretorio_pasta = os.path.join(diretorio_base_treino, personagem)

    # Verificar se a pasta existe
    if os.path.exists(diretorio_pasta):
        # Criar a pasta de destino se não existir
        caminho_destino_personagem = os.path.join(caminho_destino, personagem)
        os.makedirs(caminho_destino_personagem, exist_ok=True)

        # Redimensionar e rotacionar imagens
        redimensionar_e_rotacionar_imagens(diretorio_pasta, personagem, caminho_destino_personagem)

Contabilizando o resultados das modificações na pasta de treinamento 

In [16]:
# Diretório onde estão as pastas dos personagens
diretorio_base_treino = r'C:\Users\Pichau\Desktop\Faculdade\Projects\Facul Repositorios\Repositorios_Universitarios\Redes Neurais\Projetao\dataset_treino'

# Obtendo todas as subpastas (personagens) e seus nomes
subpastas = [(f.path, f.name) for f in os.scandir(diretorio_base_treino) if f.is_dir()]

# Contagem de imagens em cada subpasta
contagem_imagens_por_personagem = {}
for subpasta, nome_pasta in subpastas:
    arquivos = os.listdir(subpasta)
    contagem_imagens_por_personagem[nome_pasta] = len([arquivo for arquivo in arquivos if arquivo.endswith('.jpg') or arquivo.endswith('.png')])

# Ordenar as subpastas pelos números de imagens e pegar os 10 primeiros
top_10_personagens = Counter(contagem_imagens_por_personagem).most_common(10)

# Criar DataFrame pandas
df = pd.DataFrame(top_10_personagens, columns=['Personagem', 'Quantidade de Imagens'])

# Exibir DataFrame como tabela
print(df)


                 Personagem  Quantidade de Imagens
0             homer_simpson                   6738
1              ned_flanders                   4362
2               moe_szyslak                   4356
3              lisa_simpson                   4062
4              bart_simpson                   4026
5             marge_simpson                   3873
6          krusty_the_clown                   3618
7         principal_skinner                   3582
8  charles_montgomery_burns                   3579
9       milhouse_van_houten                   3237
